In [77]:
##########################################################################

# I acknowledge the use of ChatGPT and AI Tools in Helping me Code and Debug the work in this File

##########################################################################

In [ ]:
import numpy as np
import pandas as pd 
import re
import matplotlib.pyplot as plt

# file_path = "/scratch/network/mm4846/transcripts_2006_2025_03_18_final_master_v2.csv"

# df_master = pd.read_csv(file_path)

# print(df_master)
file_path = "/scratch/network/mm4846/transcripts_2006_2025_04_02_DLTR.csv"

df_dltr_corrected.to_csv(file_path)


In [76]:
(df_dltr_corrected)

,ticker,assigned_quarter,presentation,question_answer_pairs,sector_name_mapped,group_name_mapped,questions_distinct_vector,presentation_distinct_vector
0,DLTR,2024Q4,"\nGood morning, and thank you for joining us t...",[{'question': 'I wanted to start with just the...,Consumer Staples,Consumer Staples Distribution & Retail,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 1, ...","[0, 2, 4, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, ..."


In [35]:
df_dltr = pd.DataFrame({
    'ticker': ['DLTR'],
    'assigned_quarter': ['2024Q4'],
    'presentation': [pres], 
    'question_answer_pairs': [transcript_qna],  # wrap in list to avoid multiple rows
    'sector_name_mapped': ['Consumer Staples'],
    'group_name_mapped': ['Consumer Staples Distribution & Retail']    
})

In [38]:
df_dltr['question_answer_pairs'][0]

[{'question': "I wanted to start with just the tariff side and the mitigation. So it sounds like $20 million a month or about $0.85 a share is not in guidance. Can you talk about the potential mitigation of that? What are the efforts that you currently have on deck in order to see -- to mitigate that, your confidence level around your ability to offset it? And then as part of this, Mike, it does seem like you have new price points on deck, just from what we can see, $1.50, $1.75. I don't know if that's a test or not. But how important could that be to offsetting this as well? I'm just curious as to how you put all this together and you view your opportunity to mitigate what potentially could be ahead.",
  'answer': 'yes'},
 {'question': "So Mike, I think the fact that Dollar Tree quantified the 90% mitigation of the first 10% from China and the fact that the second 10% from China is currently being collected, is being assumed by the market that, that's a cost that should be embedded in

In [ ]:
# 

In [7]:
import swifter

In [39]:
import ast
import re
import pandas as pd
import numpy as np
import swifter  # For automatic parallelization

def count_mentions(text, compiled_patterns):
    """Counts occurrences of compiled regex patterns in text."""
    if not isinstance(text, str):
        return 0
    text_lower = text.lower()
    return sum(len(pattern.findall(text_lower)) for pattern in compiled_patterns)

def count_qa_mentions(qna_pairs, compiled_patterns):
    """Return total mentions in questions and answers."""
    if not isinstance(qna_pairs, list):
        return 0, 0  # Avoid unnecessary looping

    question_mentions = sum(
        count_mentions(pair.get("question", ""), compiled_patterns) for pair in qna_pairs if isinstance(pair, dict)
    )
    answer_mentions = sum(
        count_mentions(pair.get("answer", ""), compiled_patterns) for pair in qna_pairs if isinstance(pair, dict)
    )

    return question_mentions, answer_mentions

def count_qa_mentions_pairs(qna_pairs, compiled_patterns):
    """Return (#pairs_with_mention_in_question, #pairs_with_mention_in_answer)."""
    if not isinstance(qna_pairs, list):
        return 0, 0  # Avoid unnecessary looping

    question_pairs_count = sum(
        count_mentions(pair.get("question", ""), compiled_patterns) > 0 for pair in qna_pairs if isinstance(pair, dict)
    )
    answer_pairs_count = sum(
        count_mentions(pair.get("answer", ""), compiled_patterns) > 0 for pair in qna_pairs if isinstance(pair, dict)
    )

    return question_pairs_count, answer_pairs_count

def compile_patterns(word_list):
    """Precompiles regex patterns for efficiency."""
    return [re.compile(r'\b' + re.escape(term.lower()) + r'\b') for term in word_list]

def add_topic_mention_columns(df, prefix, compiled_patterns, original_word_list):
    qa_counts = np.stack(df["question_answer_pairs_parsed"].swifter.apply(
        lambda pairs: count_qa_mentions_pairs(pairs, compiled_patterns)
    ).to_numpy())

    df[f"{prefix}_mentions_qna_question_count"] = qa_counts[:, 0]

    # Correct: Use original word list directly
    df[f"{prefix}_presentation_distinct_mentions"] = df["presentation"].swifter.apply(
        lambda text: count_distinct_mentions_in_text(text, original_word_list)
    )

    return df


def count_distinct_mentions_in_text(text, topic_terms, cooldown_distance=120):
    """Count distinct bursts of topic mentions using a cooldown threshold in word distance."""
    if not isinstance(text, str):
        return 0

    text = text.lower()
    words = re.findall(r'\w+', text)  # Basic tokenizer
    word_positions = []

    # Collect positions where a topic term starts
    for i in range(len(words)):
        for term in topic_terms:
            term_words = term.lower().split()
            if words[i:i + len(term_words)] == term_words:
                word_positions.append(i)
                break  # Avoid counting overlapping terms multiple times

    # Group positions by cooldown logic
    if not word_positions:
        return 0

    mentions = 1
    last_pos = word_positions[0]
    for pos in word_positions[1:]:
        if pos - last_pos > cooldown_distance:
            mentions += 1
            last_pos = pos

    return mentions



def parse_stringified_list(df, column_name="question_answer_pairs"):
    """Ensures column contains a list of dictionaries. Parses only if the row is a string."""
    def safe_eval(val):
        if isinstance(val, list):  # Already parsed
            return val
        try:
            return ast.literal_eval(val) if isinstance(val, str) else []
        except (ValueError, SyntaxError):
            return []  # Return empty if parsing fails
    
    parsed_column = "negative_questions_parsed" if column_name == "negative_questions" else "question_answer_pairs_parsed"
    
    # Use parallelized `.swifter.apply()` for fast processing
    df[parsed_column] = df[column_name].swifter.apply(safe_eval)

    return df

def process_topic_mentions(df, word_lists_with_prefixes):
    """
    Processes multiple word lists systematically.
    Optimized with precompiled regex and parallelized operations.
    """
    compiled_patterns_dict = {prefix: compile_patterns(word_list) for prefix, word_list in word_lists_with_prefixes.items()}
    for prefix, compiled_patterns in compiled_patterns_dict.items():
        df = add_topic_mention_columns(df, prefix, compiled_patterns, word_lists_with_prefixes[prefix])

    return df


In [40]:
# df_beginning = df.iloc[:20]
# df_master['question_answer_pairs'] = df_master['qna_pairs']
df_dltr['question_answer_pairs_parsed'] = df_dltr['question_answer_pairs']
# df_dltr = parse_stringified_list(df_dltr)
df_dltr = process_topic_mentions(df_dltr, word_lists_with_prefixes)

Pandas Apply: 100%|██████████| 1/1 [00:00<00:00, 17.28it/s]


In [64]:
df_dltr['tariffs_mentions_qna_question_count']


# mention_cols = [col for col in df_dltr.columns if col.endswith('mentions_qna_question_count')]

# mention_mapping = {col.replace('mentions_qna_question_count', ''): col for col in mention_cols}
# mention_cols = sorted(mention_cols)

# # Step 2: Create the vector column
# df_dltr['questions_distinct_vector'] = df_dltr[mention_cols].values.tolist()

questions_distinct_vector = [
    df_dltr.get(topic + '_mentions_qna_question_count', pd.Series([0])).iloc[0]
    for idx, topic in sorted(index_to_topic_dict.items())
]

# Now create your DataFrame with the correct single row
df_dltr_corrected = pd.DataFrame({
    'ticker': ['DLTR'],
    'assigned_quarter': ['2024Q4'],
    'presentation': [pres], 
    'question_answer_pairs': [transcript_qna],  
    'sector_name_mapped': ['Consumer Staples'],
    'group_name_mapped': ['Consumer Staples Distribution & Retail'],
    'questions_distinct_vector': [questions_distinct_vector]
})

# Example usage:
# print(mention_cols)         # Show first 5 column names
# print(mention_mapping['ai'])    # Get the full column name for 'ai'

# inverse_mapping = {v: k for k, v in mention_mapping.items()}



['ai_presentation_distinct_mentions', 'buyback_presentation_distinct_mentions', 'capex_presentation_distinct_mentions', 'climate_presentation_distinct_mentions', 'commodity_price_presentation_distinct_mentions', 'consumer_demand_presentation_distinct_mentions', 'currency_fluctuations_hedging_presentation_distinct_mentions', 'cybersecurity_data_protection_presentation_distinct_mentions', 'debt_presentation_distinct_mentions', 'dividend_presentation_distinct_mentions', 'esg_sustainability_presentation_distinct_mentions', 'geopolitical_2_presentation_distinct_mentions', 'guidance_forecasts_presentation_distinct_mentions', 'inflation_presentation_distinct_mentions', 'interest_rates_presentation_distinct_mentions', 'labor_presentation_distinct_mentions', 'liquidity_cash_flow_presentation_distinct_mentions', 'm_and_a_presentation_distinct_mentions', 'operational_presentation_distinct_mentions', 'outperform_presentation_distinct_mentions', 'pandemic_presentation_distinct_mentions', 'political

In [74]:
presentation_distinct_vector = [
    df_dltr.get(topic + '_presentation_distinct_mentions', pd.Series([0])).iloc[0]
    for idx, topic in sorted(index_to_topic_dict_pres.items())
]

# Now add this vector to your DataFrame
df_dltr_corrected['presentation_distinct_vector'] = [presentation_distinct_vector]

In [75]:
df_dltr_corrected['presentation_distinct_vector'][0]

[0,
 2,
 4,
 0,
 0,
 1,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 9,
 0,
 0,
 0,
 0,
 4,
 0,
 3,
 5,
 2]

In [73]:
df_dltr.columns

Index(['ticker', 'assigned_quarter', 'presentation', 'question_answer_pairs',
       'sector_name_mapped', 'group_name_mapped',
       'question_answer_pairs_parsed', 'buyback_mentions_qna_question_count',
       'buyback_presentation_distinct_mentions',
       'pandemic_mentions_qna_question_count',
       'pandemic_presentation_distinct_mentions',
       'political_mentions_qna_question_count',
       'political_presentation_distinct_mentions',
       'dividend_mentions_qna_question_count',
       'dividend_presentation_distinct_mentions',
       'geopolitical_2_mentions_qna_question_count',
       'geopolitical_2_presentation_distinct_mentions',
       'climate_mentions_qna_question_count',
       'climate_presentation_distinct_mentions',
       'm_and_a_mentions_qna_question_count',
       'm_and_a_presentation_distinct_mentions',
       'outperform_mentions_qna_question_count',
       'outperform_presentation_distinct_mentions',
       'supply_chain_mentions_qna_question_count',
 

In [8]:
file_path = "/scratch/network/mm4846/transcripts_2006_2025_03_18_final_master_v2.csv"

df_master.to_csv(file_path)

Index(['ticker', 'date', 'assigned_quarter', 'quarter', 'year',
       'presentation_vector', 'qna_question_vector',
       'presentation_recurring_vector', 'presentation_episodic_vector',
       'presentation_less_frequent_vector', 'qna_question_recurring_vector',
       'qna_question_episodic_vector', 'qna_question_less_frequent_vector',
       'recurring_sum', 'episodic_sum', 'less_frequent_sum', 'recurring_count',
       'episodic_count', 'less_frequent_count', 'recurring_sum_qna',
       'episodic_sum_qna', 'less_frequent_sum_qna', 'recurring_count_qna',
       'episodic_count_qna', 'less_frequent_count_qna'],
      dtype='object')

Recurring Indices: [8, 1, 7, 12, 17, 18, 21, 25]
Episodic Indices: [14, 13, 24, 10, 4, 0, 23, 16, 2, 3]
Less Frequent Indices: [9, 5, 20, 15, 6, 22, 5, 19]
First row - presentation recurring vector: [0, 2, 2, 0, 0, 10, 1, 5]
First row - presentation episodic vector: [4, 0, 0, 1, 0, 0, 0, 0, 0, 0]
First row - presentation less frequent vector: [0, 0, 2, 0, 0, 0, 0, 2]
First row - qna recurring vector: [0, 0, 1, 1, 0, 3, 0, 0]
First row - qna episodic vector: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
First row - qna less frequent vector: [0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
word_lists_with_prefixes = {
    "buyback": share_buyback_bigrams,
    "pandemic": pandemic_terms,
    "political" : pol_bigrams,   
    "dividend": dividend_bigrams,
    "geopolitical_2": geopolitical_terms_2,
    "climate": climate_terms,
    "m_and_a": ma_spin_terms, 
    "outperform": outperform_terms,
    "supply_chain": supply_chain_terms,
    "tariffs": tariffs_terms,
    "inflation": inflation_terms,
    "capex": capex_terms, 
    "reshoring": reshoring_terms,
    "debt": debt_terms,
    "commodity_price": commodity_price_terms,
    "consumer_demand": consumer_demand_terms,
    "operational": operational_terms,
    "guidance_forecasts": guidance_forecasts_terms,
    "rd": rd_terms,
    "esg_sustainability": esg_sustainability_terms,
    "cybersecurity_data_protection": cybersecurity_data_protection_terms,
    "currency_fluctuations_hedging": currency_fluctuations_hedging_terms,
    "regulation": regulation_terms,
    "liquidity_cash_flow": liquidity_cash_flow_terms,
    "tax_strategy": tax_strategy_terms,
    "ai": ai_terms,  
    "interest_rates": interest_rate_terms,
    "labor": labor_terms 
}


In [22]:
macroeconomic_terms = [
    "interest_rates", "inflation", "currency_fluctuations_hedging", "geopolitical", "geopolitical_2",
    "consumer_demand", 
]

corporate_finance_terms = [
    "buyback", "dividend", "capex", "debt", "guidance_forecasts", "liquidity_cash_flow",  "operational", "tax_strategy"]




policy_terms = ["political", "tariffs", "regulation"]


other_trends_terms = [
"rd",    "outperform", "esg_sustainability", "ai", "pandemic", "climate",  "labor",  "rd", "labor_cost", "reshoring",
    "supply_chain", "cybersecurity_data_protection" , "commodity_price"
]

In [7]:
recurring_terms = ["buyback", "dividend", "capex", "debt",  "guidance_forecasts", "liquidity_cash_flow",  "operational",  "regulation", "tax_strategy"]
episodic_terms = ["interest_rates", "inflation", "tariffs", "geopolitical_2", 
    "consumer_demand", "ai",  "supply_chain",  "labor", "pandemic", "climate",  "commodity_price"]

less_frequent = ["esg_sustainability", "currency_fluctuations_hedging",  "rd", "labor_cost", "cybersecurity_data_protection", "reshoring", "currency_fluctuations_hedging", "outperform"]

In [54]:
index_to_topic_dict = {
    0: 'ai',
    1: 'buyback',
    2: 'capex',
    3: 'climate',
    4: 'commodity_price',
    5: 'consumer_demand',
    6: 'currency_fluctuations_hedging',
    7: 'cybersecurity_data_protection',
    8: 'debt',
    9: 'dividend',
    10: 'esg_sustainability',
    11: 'geopolitical_2',
    12: 'geopolitical',
    13: 'guidance_forecasts',
    14: 'inflation',
    15: 'interest_rates',
    16: 'labor_cost',
    17: 'labor',
    18: 'liquidity_cash_flow',
    19: 'm_and_a',
    20: 'operational',
    21: 'other',
    22: 'outperform',
    23: 'pandemic',
    24: 'political',
    25: 'rd',
    26: 'regulation',
    27: 'reshoring',
    28: 'supply_chain',
    29: 'tariffs',
    30: 'tax_strategy'
}

In [67]:
index_to_topic_dict_pres = {0: 'ai',
 1: 'buyback',
 2: 'capex',
 3: 'climate',
 4: 'commodity_price',
 5: 'consumer_demand',
 6: 'currency_fluctuations_hedging',
 7: 'cybersecurity_data_protection',
 8: 'debt',
 9: 'dividend',
 10: 'esg_sustainability',
 11: 'geopolitical_2',
 12: 'guidance_forecasts',
 13: 'inflation',
 14: 'interest_rates',
 15: 'labor',
 16: 'liquidity_cash_flow',
 17: 'm_and_a',
 18: 'operational',
 19: 'outperform',
 20: 'pandemic',
 21: 'political',
 22: 'rd',
 23: 'regulation',
 24: 'reshoring',
 25: 'supply_chain',
 26: 'tariffs',
 27: 'tax_strategy'}


In [32]:


sector_map = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",
    25: "Consumer Discretionary",
    30: "Consumer Staples",
    35: "Health Care",
    40: "Financials",
    45: "Information Technology",
    50: "Communication Services",
    55: "Utilities",
    60: "Real Estate"
}
industry_group_map = {
    1010: "Energy",
    1510: "Materials",
    2010: "Capital Goods",
    2020: "Commercial & Professional Services",
    2030: "Transportation",
    2510: "Automobiles & Components",
    2520: "Consumer Durables & Apparel",
    2530: "Consumer Services",
    2550: "Consumer Discretionary Distribution & Retail",
    3010: "Consumer Staples Distribution & Retail",
    3020: "Food, Beverage & Tobacco",
    3030: "Household & Personal Products",
    3510: "Health Care Equipment & Services",
    3520: "Pharmaceuticals, Biotechnology & Life Sciences",
    4010: "Banks",
    4020: "Financial Services",
    4030: "Insurance",
    4510: "Software & Services",       # GICS uses 4510 for the entire group,
    #  -- note that GICS also further splits 4510 into 451020, 451030 for Industries
    4510: "Software & Services",       # repeated key is typically not needed, see details below
    4520: "Technology Hardware & Equipment",
    4530: "Semiconductors & Semiconductor Equipment",
    5010: "Telecommunication Services",
    5020: "Media & Entertainment",
    5510: "Utilities",
    6010: "Equity Real Estate Investment Trusts (REITs)",
    6020: "Real Estate Management & Development"
}



industry_map = {
    # -- Sector 10: Energy
    101010: "Energy Equipment & Services",
    101020: "Oil, Gas & Consumable Fuels",
    
    # -- Sector 15: Materials
    151010: "Chemicals",
    151020: "Construction Materials",
    151030: "Containers & Packaging",
    151040: "Metals & Mining",
    151050: "Paper & Forest Products",
    
    # -- Sector 20: Industrials
    201010: "Aerospace & Defense",
    201020: "Building Products",
    201030: "Construction & Engineering",
    201040: "Electrical Equipment",
    201050: "Industrial Conglomerates",
    201060: "Machinery",
    201070: "Trading Companies & Distributors",
    202010: "Commercial Services & Supplies",
    202020: "Professional Services",
    203010: "Air Freight & Logistics",
    203020: "Passenger Airlines",
    203030: "Marine Transportation",
    203040: "Ground Transportation",
    203050: "Transportation Infrastructure",
    
    # -- Sector 25: Consumer Discretionary
    251010: "Automobile Components",
    251020: "Automobiles",
    252010: "Household Durables",
    252020: "Leisure Products",
    252030: "Textiles, Apparel & Luxury Goods",
    253010: "Hotels, Restaurants & Leisure",
    253020: "Diversified Consumer Services",
    255010: "Distributors",
    255030: "Broadline Retail",
    255040: "Specialty Retail",
    
    # -- Sector 30: Consumer Staples
    301010: "Consumer Staples Distribution & Retail",
    302010: "Beverages",
    302020: "Food Products",
    302030: "Tobacco",
    303010: "Household Products",
    303020: "Personal Care Products",
    
    # -- Sector 35: Health Care
    351010: "Health Care Equipment & Supplies",
    351020: "Health Care Providers & Services",
    351030: "Health Care Technology",
    352010: "Biotechnology",
    352020: "Pharmaceuticals",
    352030: "Life Sciences Tools & Services",
    
    # -- Sector 40: Financials
    401010: "Banks",
    402010: "Financial Services",
    402020: "Consumer Finance",
    402030: "Capital Markets",
    402040: "Mortgage REITs",
    403010: "Insurance",
    
    # -- Sector 45: Information Technology
    451020: "IT Services",
    451030: "Software",
    452010: "Communications Equipment",
    452020: "Technology Hardware, Storage & Peripherals",
    452030: "Electronic Equipment, Instruments & Components",
    453010: "Semiconductors & Semiconductor Equipment",
    
    # -- Sector 50: Communication Services
    501010: "Diversified Telecommunication Services",
    501020: "Wireless Telecommunication Services",
    502010: "Media",
    502020: "Entertainment",
    502030: "Interactive Media & Services",
    
    # -- Sector 55: Utilities
    551010: "Electric Utilities",
    551020: "Gas Utilities",
    551030: "Multi-Utilities",
    551040: "Water Utilities",
    551050: "Independent Power and Renewable Electricity Producers",
    
    # -- Sector 60: Real Estate
    601010: "Diversified REITs",
    601025: "Industrial REITs",
    601030: "Hotel & Resort REITs",
    601040: "Office REITs",
    601050: "Health Care REITs",
    601060: "Residential REITs",
    601070: "Retail REITs",
    601080: "Specialized REITs",
    602010: "Real Estate Management & Development"
}


In [33]:

subindustry_map = {
    # -- Energy (10 -> 101010, 101020)
    10101010: "Oil & Gas Drilling",
    10101020: "Oil & Gas Equipment & Services",
    10102010: "Integrated Oil & Gas",
    10102020: "Oil & Gas Exploration & Production",
    10102030: "Oil & Gas Refining & Marketing",
    10102040: "Oil & Gas Storage & Transportation",
    10102050: "Coal & Consumable Fuels",
    
    # -- Materials (15 -> e.g. 151010, 151020, 151030, 151040, 151050)
    15101010: "Commodity Chemicals",
    15101020: "Diversified Chemicals",
    15101030: "Fertilizers & Agricultural Chemicals",
    15101040: "Industrial Gases",
    15101050: "Specialty Chemicals",
    15102010: "Construction Materials",
    15103010: "Metal, Glass & Plastic Containers",
    15103020: "Paper & Plastic Packaging Products & Materials",
    15104010: "Aluminum",
    15104020: "Diversified Metals & Mining",
    15104025: "Copper",
    15104030: "Gold",
    15104040: "Precious Metals & Minerals",
    15104045: "Silver",
    15104050: "Steel",
    15105010: "Forest Products",
    15105020: "Paper Products",
    
    # -- Industrials (20 -> e.g. 20101010, 20102010, etc.)
    20101010: "Aerospace & Defense",
    20102010: "Building Products",
    20103010: "Construction & Engineering",
    20104010: "Electrical Components & Equipment",
    20104020: "Heavy Electrical Equipment",
    20105010: "Industrial Conglomerates",
    20106010: "Construction Machinery & Heavy Transportation Equipment",
    20106015: "Agricultural & Farm Machinery",
    20106020: "Industrial Machinery & Supplies & Components",
    20107010: "Trading Companies & Distributors",
    20201010: "Commercial Printing",
    20201050: "Environmental & Facilities Services",
    20201060: "Office Services & Supplies",
    20201070: "Diversified Support Services",
    20201080: "Security & Alarm Services",
    20202010: "Human Resource & Employment Services",
    20202020: "Research & Consulting Services",
    20202030: "Data Processing & Outsourced Services",
    20301010: "Air Freight & Logistics",
    20302010: "Passenger Airlines",
    20303010: "Marine Transportation",
    20304010: "Rail Transportation",
    20304030: "Cargo Ground Transportation",
    20304040: "Passenger Ground Transportation",
    20305010: "Airport Services",
    20305020: "Highways & Railtracks",
    20305030: "Marine Ports & Services",
    
    # -- Consumer Discretionary (25)
    25101010: "Automotive Parts & Equipment",
    25101020: "Tires & Rubber",
    25102010: "Automobile Manufacturers",
    25102020: "Motorcycle Manufacturers",
    25201010: "Consumer Electronics",
    25201020: "Home Furnishings",
    25201030: "Homebuilding",
    25201040: "Household Appliances",
    25201050: "Housewares & Specialties",
    25202010: "Leisure Products",
    25203010: "Apparel, Accessories & Luxury Goods",
    25203020: "Footwear",
    25203030: "Textiles",
    25301010: "Casinos & Gaming",
    25301020: "Hotels, Resorts & Cruise Lines",
    25301030: "Leisure Facilities",
    25301040: "Restaurants",
    25302010: "Education Services",
    25302020: "Specialized Consumer Services",
    25501010: "Distributors",
    25503030: "Broadline Retail",
    25504010: "Apparel Retail",
    25504020: "Computer & Electronics Retail",
    25504030: "Home Improvement Retail",
    25504040: "Other Specialty Retail",
    25504050: "Automotive Retail",
    25504060: "Homefurnishing Retail",
    
    # -- Consumer Staples (30)
    30101010: "Drug Retail",
    30101020: "Food Distributors",
    30101030: "Food Retail",
    30101040: "Consumer Staples Merchandise Retail",
    30201010: "Brewers",
    30201020: "Distillers & Vintners",
    30201030: "Soft Drinks & Non-alcoholic Beverages",
    30202010: "Agricultural Products & Services",
    30202030: "Packaged Foods & Meats",
    30203010: "Tobacco",
    30301010: "Household Products",
    30302010: "Personal Care Products",
    
    # -- Health Care (35)
    35101010: "Health Care Equipment",
    35101020: "Health Care Supplies",
    35102010: "Health Care Distributors",
    35102015: "Health Care Services",
    35102020: "Health Care Facilities",
    35102030: "Managed Health Care",
    35103010: "Health Care Technology",
    35201010: "Biotechnology",
    35202010: "Pharmaceuticals",
    35203010: "Life Sciences Tools & Services",
    
    # -- Financials (40)
    40101010: "Diversified Banks",
    40101015: "Regional Banks",
    40201020: "Diversified Financial Services",
    40201030: "Multi-Sector Holdings",
    40201040: "Specialized Finance",
    40201050: "Commercial & Residential Mortgage Finance",
    40201060: "Transaction & Payment Processing Services",
    40202010: "Consumer Finance",
    40203010: "Asset Management & Custody Banks",
    40203020: "Investment Banking & Brokerage",
    40203030: "Diversified Capital Markets",
    40203040: "Financial Exchanges & Data",
    40204010: "Mortgage REITs",
    40301010: "Insurance Broker",
    40301020: "Life & Health Insurance",
    40301030: "Multi-line Insurance",
    40301040: "Property & Casualty Insurance",
    40301050: "Reinsurance",
    
    # -- Information Technology (45)
    45102010: "IT Consulting & Other Services",
    45102030: "Internet Services & Infrastructure",
    45103010: "Application Software",
    45103020: "Systems Software",
    45201020: "Communications Equipment",
    45202030: "Technology Hardware, Storage & Peripherals",
    45203010: "Electronic Equipment & Instruments",
    45203015: "Electronic Components",
    45203020: "Electronic Manufacturing Services",
    45203030: "Technology Distributors",
    45301010: "Semiconductor Materials & Equipment",
    45301020: "Semiconductors",
    
    # -- Communication Services (50)
    50101010: "Alternative Carriers",
    50101020: "Integrated Telecommunication Services",
    50102010: "Wireless Telecommunication Services",
    50201010: "Advertising",
    50201020: "Broadcasting",
    50201030: "Cable & Satellite",
    50201040: "Publishing",
    50202010: "Movies & Entertainment",
    50202020: "Interactive Home Entertainment",
    50203010: "Interactive Media & Services",
    
    # -- Utilities (55)
    55101010: "Electric Utilities",
    55102010: "Gas Utilities",
    55103010: "Multi-Utilities",
    55104010: "Water Utilities",
    55105010: "Independent Power Producers & Energy Traders",
    55105020: "Renewable Electricity",
    
    # -- Real Estate (60)
    60101010: "Diversified REITs",
    60102510: "Industrial REITs",
    60103010: "Hotel & Resort REITs",
    60104010: "Office REITs",
    60105010: "Health Care REITs",
    60106010: "Multi-Family Residential REITs",
    60106020: "Single-Family Residential REITs",
    60107010: "Retail REITs",
    60108010: "Other Specialized REITs",
    60108020: "Self-Storage REITs",
    60108030: "Telecom Tower REITs",
    60108040: "Timber REITs",
    60108050: "Data Center REITs",
    60201010: "Diversified Real Estate Activities",
    60201020: "Real Estate Operating Companies",
    60201030: "Real Estate Development",
    60201040: "Real Estate Services"
}


In [3]:
# pol_text = [
#     "prescription drug", "cut medicare", "government takeover", "drug plan",
#     "for lowincome", "health care", "human services", "medicare prescription",
#     "have health", "generic drugs", "schip benefits", "like medicaid", 
#     "provide health", "of health", "health insurance",

#     "air act", "from renewable", "climate change", "clean air", "states rights",
#     "greenhouse gas", "nuclear power", "nations energy", "foreign oil", "with opec",
#     "global warming", "energy independence", "legislators are", "emissions from", 
#     "carbon emissions",

#     "estate tax", "tax relief", "bush tax", "the estate", "middleclass tax", 
#     "continued unfair", "full repeal", "typical american", "increase taxes", 
#     "raise taxes", "tax cuts", "largest budget", "repeals the", "tax reform", 
#     "tax cut",

#     "campaign finance", "constitution to", "finance reform", "federal elections", 
#     "appropriations bills", "political system", "constitution and", "public financing", 
#     "of voters", "in politics", "on immigration", "constitution the", 
#     "presidential elections", "federal election", "political parties",

#     "minimum wage", "balanced budget", "legislation provides", "bankruptcy bill", 
#     "medicaid matching", "time congress", "emergency economic", "and discourages", 
#     "surpluses in", "in aid", "create jobs", "government when", 
#     "congress does", "waste by", "government taxes",

#     "free trade", "trade agreement", "trade agreements", "trade barriers", 
#     "freetrade agreement", "up markets", "the andean", "globalization is", 
#     "labor standards", "all trade", "policy objectives", "jordan the", 
#     "american free", "trade relations", "duties on",

#     "street station", "fairness doctrine", "cyber warfare", "on highways", 
#     "faithbased organizations", "human services", "require public", "and faithbased", 
#     "proposals during", "private entities", "the fcc", "structurally deficient", 
#     "hightech jobs", "highspeed rail", "every government",

#     "on terror", "from iraq", "bin laden", "nuclear weapons", "our troops", 
#     "commander in", "in chief", "al qaeda", "weapons of", "mass destruction", 
#     "of military", "in afghanistan", "constitution to", "osama bin", "to authorize"
# ]



pol_bigrams = [
    "the constitution",
    "the states",
    "public opinion",
    "interest groups",
    "of government",
    "the GOP",
    "in Congress",
    "national government",
    "social policy",
    "the civil",
    "elected officials",
    "politics is",
    "political parties",
    "office of",
    "the political",
    "interest group",
    "the bureaucracy",
    "and Senate",
    "government and",
    "for governor",
    "executive branch",
    "support for",
    "the EPA",
    "in government",
    "Congress to",
    "political process",
    "care reform",
    "government in",
    "due process",
    "President Obama",
    "and social",
    "first amendment",
    "Congress the",
    "the Republican",
    "Tea Party",
    "the legislative",
    "of civil",
    "court has",
    "groups and",
    "struck down",
    "shall have",
    "civil war",
    "the Congress",
    "the constitutional",
    "politics and",
    "ruled that",
    "the South",
    "the presidential",
    "government is",
    "of representatives",
    "yes vote",
    "policy goals",
    "to enact",
    "African Americans",
    "economic policy",
    "proposed by",
    "of social",
    "a political",
    "of speech",
    "federal bureaucracy",
    "civil service",
    "and party",
    "government policy",
    "federal courts",
    "state the",
    "argued that",
    "executive privilege",
    "the democratic",
    "of politics",
    "islamic state",
    "president has",
    "national security",
    "the candidates",
]


share_buyback_bigrams = [
    "share buyback",
    "stock repurchase",
    "repurchase authorization",
    "buyback program",
    "repurchase plan",
    "authorized repurchase",
    "stock buyback",
    "shareholder return",
    "buyback activity",
    "repurchase activity",
    "capital return",
    "buyback announcement",
    "share repurchase",
    "repurchase initiative",
    "repurchase schedule",
    "buyback",
    "repurchase",
    "repo",
    "share repo",
]



dividend_bigrams = [
    "dividend",
    "dividend payout",
    "dividend increase",
    "payout ratio",
    "dividend yield",
    "quarterly dividend",
    "annual dividend",
    "special dividend",
    "dividend declaration",
    "dividend growth",
    "dividend payment",
    "cash dividend",
    "dividend policy",
    "declared dividend",
    "dividend distribution",
    "dividend reinvestment"
]




geopolitical_terms = [
    "war", "military", "terror", "terrorism", "terrorist", "geopolitical",
    "conflict", "Middle East", "Iraq", "Afghanistan", "Iran", "Syria", "Libya",
    "Ukraine", "Ukrainian", "Russia", "Russian", "North Korea", "Venezuela",
    "coup", "expropriation", "confiscation", "nationalism", "security",
    "protest", "protests", "country", "countries", "political", "retaliation",
    "unrest", "geographic", "geography", "troop", "troops", "sanction",
    "sanctions", "embargo", "wars", "warfare", "army", "navy", "weapon",
    "weapons", "combat", "missile", "missiles", "immigration", "diplomacy"
]



climate_terms = [
    "air pollution", "electric vehicle", "new energy",
    "air quality", "energy climate", "ozone layer",
    "air temperature", "energy conversion", "renewable energy",
    "biomass energy", "energy efficient", "sea level",
    "carbon dioxide", "energy environment", "sea water",
    "carbon emission", "environmental sustainability", "snow ice",
    "carbon energy", "extreme weather", "solar energy",
    "carbon neutral", "flue gas", "solar thermal",
    "carbon price", "forest land", "sustainable energy",
    "carbon sink", "gas emission", "water resource",
    "carbon tax", "ghg emission", "water resources",
    "clean air", "global decarbonization", "wave energy",
    "clean energy", "global warm", "weather climate",
    "clean water", "greenhouse gas", "wind energy",
    "climate change", "heat power", "wind power",
    "coastal area", "Kyoto protocol", "wind resource",
    "coastal region", "natural hazard",
    "greenhouse gas", "gas emission", "carbon tax", "emission trade", "carbon reduction",
    "reduce emission", "air pollution", "carbon price", "dioxide emission", "carbon market",
    "carbon emission", "reduce carbon", "environmental standard", "epa regulation", "mercury emission",
    "carbon dioxide", "energy regulatory", "nox emission", "energy independence",
    "coastal area", "forest land", "storm water", "natural hazard", "water discharge",
    "global warm", "sea level", "heavy snow", "sea water", "ice product",
    "snow ice", "nickel metal", "air water", "warm climate"
]

pandemic_terms = [
    "sarscov",
    "coronavirus",
    "corona virus",
    "ncov",
    "covid",
    "hn", 
    "swine flu",
    "ahn",
    "sars",
    "merscov",
    "severe acute respiratory syndrome",
    "middle east respiratory syndrome",
    "mers",
    "ebola",
    "zika"
]

outperform_terms = [
"beat expectations",
    "outpeformed",
    "outperform",
    "outperforming",
"exceeded guidance",
"outperformed peers",
"strong growth",
"record quarter",     
    "record sales",
"surpassed estimates", 
    "beat estimates",
"margin expansion"
"strong momentum",
"improved performance",
"strong quarter"
]


supply_chain_terms = [
"supply disruptions", 
"logistics delays",
"material shortages",
"shipping constraints",
"inventory bottlenecks",
"freight challenges",
"supplier delays",   
"distribution issues",
"procurement challenges",  
"supply chain constraints", 
"supply chain", 
]

tariffs_terms = [
    "tariff",
    "tariffs",
    "tariff impact",
    "tariff increases",
    "import duties",
    "import duty",
    "trade barriers",
    "trade war",
    "retaliatory tariffs",
    "customs duty",
    "protectionism",
    "import taxes",
    "levies",
    "section 301",
    "section 232",
#     "exemptions",
    "tariff relief",
    "anti-dumping",
    "countervailing duty",
    "trade disputes",
    "border tax",
#     "compliance cost",
    "export restrictions",
    "duty drawback",
    "import penalties",
    "import penalty",
    "import tax"    
]


inflation_terms = [
    "inflation",
    "inflationary pressure",
    "inflationary environment",
    "inflation outlook",
    "price increases",
    "pricing pressure",
    "cost escalation",
    "cost inflation",
    "input cost",
    "commodity prices",
    "commodity inflation",
    "energy inflation",
    "wage inflation",
    "labor costs",
    "raw material costs",
    "pricing actions",
    "pricing power",
#     "margin compression",
    "cost pass-through",
#     "interest rates",
    "monetary tightening",
    "headline inflation",
    "core inflation",
    "CPI reading",
    "producer prices",
#     "macroeconomic headwinds",
#     "economic pressure",
    "purchasing power",
#     "demand elasticity",
    "cost environment"
]


capex_terms = [
    "capital expenditure",
    "capital expenditures",
    "capex",
    "capital spending",
    "investment in capacity",
    "plant expansion",
    "equipment upgrade",
    "capital allocation",
    "maintenance capital",
    "capital projects",
    "expansion plans",
    "capital budget",
    "infrastructure investment",
]


labor_cost_terms = [
    "labor costs",
    "wage pressure",
    "wage inflation",
    "workforce expenses",
    "employee benefits",
    "labor negotiations",
    "union contracts",
    "payroll costs",
    "talent shortage",
    "staffing challenges",
    "overtime expenses",
    "minimum wage",
]


reshoring_terms = [
    "reshoring",
    "onshoring",
    "nearshoring",
    "domestic production",
    "local manufacturing",
    "supply chain relocation",
    "manufacturing footprint",
    "domestic supply base",
    "production shift",
    "localized operations",
    "plant repatriation",
]



debt_terms = [
    "debt",
    "leverage",
    "bond issuance",
    "credit facility",
    "interest expense",
    "loan covenants",
    "refinancing",
    "debt maturity",
    "capital structure",
    "liquidity position",
    "net debt",
    "balance sheet",
    "leverage ratio",
    "debt repayment",
    "interest coverage",
]

commodity_price_terms = [
    "commodity prices",
    "raw material costs",
    "input costs",
    "steel prices",
    "oil price",
    "energy costs",
    "agricultural commodities",
    "metal prices",
    "commodity hedge",
    "commodity volatility",
    "spot price",
    "commodity futures",
    "commodity price",
    "oil futures",
    "grain prices",
    "corn prices",
    "wheat prices",
    "soybean prices",
    "crop prices",
    "fertilizer costs",
    "brent crude",
    "wti crude",
    "natural gas prices",
    "gasoline prices",
    "fuel costs",
    "diesel prices",
    "aluminum prices",
    "copper prices",
    "lumber prices",
    "nickel prices",
    "zinc prices",
    "precious metals",
    "input inflation",
    "commodity inflation",
    "materials inflation",
    "packaging costs",
    "freight costs",
    "commodity index",
    "commodity-linked",
    "commodity exposure",
    "hedging costs",
    "futures curve",
    "backwardation",
    "contango",
    "commodity pricing",
    "input cost inflation"
]


geopolitical_terms_2 = [
    "geopolitical",
    "war",
    "military conflict",
    "sanctions",
    "country risk",
    "political unrest",
    "security issues",
    "embargo",
    "diplomatic tensions",
    "international relations",
    "expropriation",
    "terrorism",
]

consumer_demand_terms = [
    "consumer demand",
    "consumer spending",
    "customer orders",
    "purchasing behavior",
    "demand outlook",
    "sales momentum",
    "foot traffic",
    "customer sentiment",
    "end-market demand",
    "buying patterns",
    "consumer trends",
    "order backlog",
    "customer demand",
    "consumer behavior",
    "customer purchasing patterns",
    "retail demand",
    "wholesale demand",
    "demand environment",
    "customer activity",
    "consumer activity",
    "sales volumes",
    "purchase intent",
    "booking trends",
    "customer buying behavior",
    "purchase behavior",
    "order volume",
    "order trends",
    "point-of-sale data",
    "same-store sales",
    "basket size",
    "repeat purchases",
    "transaction volume"
]

operational_terms = [
    # Core operational phrases
    "operational efficiency",
    "operational improvement",
    "operational excellence",
    "operational metrics",
    "operating performance",
    "operational synergies",
    "process optimization",
    "throughput",
    "production capacity",
    "capacity utilization",
    "asset utilization",
    "downtime",
    "maintenance shutdown",
    "operational risk",
    "workflow efficiency",
    "standard operating procedures",
    "cycle time",
    "lead time",
    "yield improvement",
    "quality control",
    "quality assurance",
    "efficiency",

    # Cost & margin management
    "cost structure",
    "cost optimization",
    "cost discipline",
    "cost savings program",
    "cost-cutting initiatives",
    "overhead reduction",
    "fixed cost",
    "fixed costs", 
    "variable costs", 
    "variable cost",
    "operating expenses",
    "operating leverage",
    "OPEX",
    "SG&A",
    "COGS", 
    "margin expansion",
    "margin compression",
    "gross margin",
    "operating margin",
    "EBIT margin",
    "cost escalation",
    
    # Revenue & profit
    "revenue",
    "sales growth",
    "top-line growth",
    "bottom-line results",
    "operating income",
    "operating profit",
    "EBIT",
    "EBITDA",
    "net income",
    "profitability metrics",
    "profit margin",
    "revenue growth",
    "sales revenue",
    "income statement highlights",

    # Production & capacity
    "production ramp-up",
    "production volume",
    "production efficiency",
    "capacity expansion",
    "facility utilization",
    "bottleneck reduction",
    "line capacity",
    "scaling operations",
    "manufacturing throughput",
    "plant expansion",
    "factory optimization",
    "assembly line efficiency",

    # Supply & inventory
    "inventory management",
    "inventory turnover",
    "inventory optimization",
    "stock levels",
    "lead-time reduction",
    "logistics efficiency",
    "distribution network",
    "warehousing strategy",
    "fulfillment capacity",

    # Workforce & organization
    "labor productivity",
    "workforce efficiency",
    "headcount reduction",
    "staffing optimization",
    "shift scheduling",
    "talent allocation",
    "organizational restructuring",
    "resource utilization",

    # Continuous improvement methods
    "lean manufacturing",
    "lean processes",
    "six sigma",
    "continuous improvement",
    "Kaizen",
    "best practices",
    "process reengineering",
    "productivity gains",
    "efficiency ratio",

    # Synergies & integration (especially post-M&A)
    "synergy realization",
    "synergy capture",
    "integration plan",
    "cost synergies",
    "operational integration",
    "merger integration milestones",

    # General operational commentary
    "operational highlights",
    "operational focus",
    "business continuity",
    "operational resiliency",
    "day-to-day operations",
    "operational challenges",
    "operational outlook",
    "execution risk",
    "scalability",
    "throughput constraints",
]




guidance_forecasts_terms = [
    "earnings guidance",
    "revenue outlook",
    "EPS outlook",
    "growth projections",
    "guidance range",
    "visibility into next quarter",
    "full-year expectations",
    "long-term targets",
    "quarterly guidance",
    "revenue expectations",
    "guidance assumptions",
    "guidance revision",
    "margin guidance",
    "guidance"
]

rd_terms = [
    "earnings guidance",
    "R&D",
    "revenue outlook",
    "EPS outlook",
    "growth projections",
    "guidance range",
    "visibility into next quarter",
    "full-year expectations",
    "long-term targets",
    "quarterly guidance",
    "revenue expectations",
    "guidance assumptions",
    "guidance revision",
    "margin guidance",
    "R&D spending",
    "innovation pipeline",
    "technology investments",
    "new product development",
    "innovation pipeline",
    "technology investments",
    "development timeline",
    "R&D expense",
    "commercialization strategy",
    "clinical trials",
    "new product development",
    "innovation pipeline",
    "technology roadmap",
    "development timeline",
    "R&D intensity",
    "research partnerships",
    "patent filings"
]

esg_sustainability_terms = [
    "emissions reduction",
    "net-zero targets",
    "carbon footprint",
    "renewable energy",
    "sustainability initiatives",
    "diversity & inclusion",
    "ESG reporting",
    "corporate social responsibility",
    "diversity targets",
    "social responsibility",
    "environmental impact",
    "governance practices",
    "diversity goals",
    "sustainability metrics",
    "corporate governance",
    "climate commitments",
    "ESG disclosures",
    "community engagement",
    "green investments",
    "climate risk",
    "social responsibility"
]

cybersecurity_data_protection_terms = [
    "cybersecurity",
    "data protection",
    "cyber attack",
    "data privacy",
    "cyber risk",
    "security investments",
    "breach prevention",
    "IT security",
    "ransomware",
    "data privacy",
    "cyber risk",
    "security investments",
    "breach prevention",
    "IT security",
    "ransomware",
    "incident response",
    "cybersecurity spending",
    "vulnerability assessment",
    "security protocols",
    "data governance",
    "regulatory compliance",
    "cyber threat landscape",
    "network security",
    "cloud security",
    "threat detection",
    "system resilience",
    "endpoint protection",
    "GDPR compliance",
    "CCPA compliance",
    "data breach",
    "privacy regulations",
    "security breach"
]

currency_fluctuations_hedging_terms = [
    "foreign exchange",
    "currency fluctuations",
    "currency hedging",
    "FX hedging",
    "currency exposure",
    "foreign exchange risk",
    "FX volatility",
    "hedging strategies",
    "FX movements",
    "exchange rate impact",
    "currency headwinds",
    "translation risk",
    "FX hedging",
    "currency fluctuations",
    "foreign revenue exposure",
    "hedging instruments",
    "currency translation effects",
    "hedge contracts",
    "currency impacts",
    "FX exposure"
]

regulation_terms = [
    "regulation", "regulatory", "compliance", "oversight", "governance", "enforcement",
     "mandate", "rulemaking", "statutory", "reform", "policy",
    "legal requirements", "regulatory landscape", "restrictions", "standards",
    "approval process", "compliance costs", "licensing", "monitoring",
    "impact of regulations", "evolving regulations", "regulatory hurdles", "government oversight",
    "audits", "reporting requirements", "regulatory challenges", "shifting regulatory environment",
    "industry standards", "regulatory burden", "penalties", "litigation risk", "policy changes",
    "new legislation", "legal exposure", "regulatory headwinds", "transparency requirements", 
    "regulatory scrutiny", "adapting to regulations", "compliance initiatives",
    "heightened oversight", "agency directives", "ongoing discussions with regulators",
    "evolving compliance landscape", "uncertainty in regulations", "regulatory clarity"
]

liquidity_cash_flow_terms = [
    "liquidity",
    "cash flow",
    "free cash flow",
    "cash generation",
    "working capital",
    "capital allocation",
    "cash balance",
    "liquidity position",
    "operating cash flow",
    "cash utilization",
    "cash deployment",
    "cash management",
    "free cash flow",
    "cash reserves",
    "cash burn rate",
    "funding needs",
    "cash generation",
    "working capital management",
    "cash cycle",
    "financing activities",
    "debt maturities"
]


tax_strategy_terms = [
    "tax strategy",
    "tax rate",
    "effective tax rate",
    "tax planning",
    "tax expense",
    "tax reform",
    "tax implications",
    "tax rate",
    "corporate tax",
    "tax policy",
    "tax benefits",
    "tax credits",
    "tax legislation",
    "tax liabilities",
    "deferred tax",
    "tax guidance",
    "tax savings",
    "international taxation",
    "tax efficiency",
    "tax optimization",
    "tax exposure",
    "transfer pricing",
    "tax incentives",
    "tax treatment",
    "tax burden",
    "tax provisions",
    "effective tax rate",
    "tax compliance",
    "tax audit",
    "tax strategy",
    "tax regime",
    "tax position",
    "tax expense",
    "tax avoidance",
    "tax risk",
    "global tax",
    "tax jurisdictions",
    "tax disputes",
    "tax management",
    "tax structure",
    "effective rate",
    "statutory rate",
    "repatriation",
    "transfer pricing",
    "offshore earnings",
    "cash repatriation",
    "foreign earnings",
    "earnings overseas",
    "jurisdictional mix",
    "offshore cash",
    "offshore profits",
]


ma_spin_terms = [
    # Mergers & Acquisitions
    "merger",
    "acquisition",
    "merger agreement",
    "strategic acquisition",
    "acquisition target",
    "acquisition strategy",
    "acquisition pipeline",
    "acquisition integration",
    "M&A activity",
    "M&A transaction",
    "deal announcement",
    "deal completion",
    "deal closing",
    "purchase agreement",
    "business combination",

    # Spin-offs & Divestitures
    "spin-off",
    "spinoff transaction",
    "divestiture",
    "business separation",
    "asset divestiture",
    "carve-out",
    "non-core divestiture",
    "portfolio simplification",
    "stand-alone entity",
    "subsidiary spin-off",
    "tax-free spin-off",
    "strategic separation",

    # Generic M&A/Spin lingo
    "transaction value",
    "sale of division",
    "divestment",
    "strategic alternatives"
]

ai_terms = [
    "artificial intelligence", "machine learning", "deep learning", "neural networks",
    "large language model", "generative AI", "automation", "computer vision", "natural language processing",
    "chatbot", "predictive analytics", "AI-driven", "AI-powered", "autonomous systems", "algorithmic trading",
    "robotics", "reinforcement learning", "AI research", "supervised learning", "unsupervised learning",
    "AI adoption", "AI infrastructure", "AI applications", "transformer models", "big data analytics",
    "data science", "speech recognition", "AI ethics", "AI regulation", "computational intelligence",
    "automated decision-making", "deep neural networks", "AI integration", "AI investment",
    "AI capabilities", "AI-powered solutions", "LLM deployment", "machine learning models",
    "AI efficiency", "AI-enhanced operations", "intelligent automation", "AI-driven insights",
    "AI cost savings", "AI monetization"
]

interest_rate_terms = [
    "interest rate", "federal funds rate", "monetary policy", "central bank", "rate hike",
    "rate cut", "yield curve", "bond yields", "credit market", "fixed income", "rate environment",
    "interest rate risk", "rate-sensitive sectors", "benchmark rate", "rate forecast", "inflation targeting",
    "real interest rates", "discount rate", "repo rate", "policy rate", "long-term rates",
    "short-term rates", "rate impact", "macroprudential policy", "sovereign debt yields", "credit spreads",
    "rate expectations", "interest rate swap", "economic tightening", "quantitative easing",
    "liquidity injections", "rate volatility", "neutral rate", "financial conditions index",
    "debt servicing costs", "cost of capital", "interest rate headwinds", "interest expense",
    "refinancing risk", "rate uncertainty", "borrowing costs", "debt markets"
]

labor_terms = [
    "labor market", "employment rate", "unemployment", "job creation", "labor force participation",
    "wage growth", "workforce development", "labor shortage", "job market", "labor unions",
    "gig economy", "minimum wage", "workforce training", "skilled labor", "blue-collar workers",
    "white-collar employment", "hiring trends", "job openings", "wage pressure", "employment trends",
    "remote work", "workforce retention", "labor mobility", "contract workers", "full-time employment",
    "part-time employment", "staffing shortages", "union negotiations", "strike activity",
    "labor disputes", "worker productivity", "job security", "employee benefits", "wage inflation",
    "labor cost management", "recruitment challenges", "hiring freeze", "layoffs",
    "employee turnover", "labor cost headwinds", "unionized workforce", "workforce flexibility", "headcount", "labor", "employees", "employee"
]










In [16]:
transcript_qna = [
    {
        'question': "I wanted to start with just the tariff side and the mitigation. So it sounds like $20 million a month or about $0.85 a share is not in guidance. Can you talk about the potential mitigation of that? What are the efforts that you currently have on deck in order to see -- to mitigate that, your confidence level around your ability to offset it? And then as part of this, Mike, it does seem like you have new price points on deck, just from what we can see, $1.50, $1.75. I don't know if that's a test or not. But how important could that be to offsetting this as well? I'm just curious as to how you put all this together and you view your opportunity to mitigate what potentially could be ahead.",
        'answer': "yes"
    },
    {
        'question': "So Mike, I think the fact that Dollar Tree quantified the 90% mitigation of the first 10% from China and the fact that the second 10% from China is currently being collected, is being assumed by the market that, that's a cost that should be embedded in the P&L for this year. So a, is that wrong? And b, can you give us a sense of what your overall sourcing portfolio looks like right now, such that when the reciprocal tariffs do come out, if they do, we can get a sense for the exposures for the core Dollar Tree, and that would help us understand the potential financial impact? And then on top of that, to what extent is Dollar Tree willing to use its balance sheet and all the cash that has on its balance sheet, especially as the Family Dollar sales consummated to offset the potential margin implications of these tariffs and support the EPS outlook from here?",
        'answer': "yes"
    },
    {
        'question': "Congratulations on the Family Dollar sale. Mike, I wanted to ask you the, I guess, philosophy of how you're going to run the business over the next few years. You have a fresh start now with one asset. And if you look back, inflation has been pretty challenging for Dollar stores. And now you still have to navigate tariffs. So I wanted to ask how you think about margins for the business. Do you invest during the next several years, keeping margin down to reinvest back in the company? You have your competitors. Your biggest one is reinvesting in itself. Not sure what Family Dollar's plans will be. But curious, do you let margins run up? Or do you have to keep a lot of ammunition and firepower given the pretty uncertain backdrop that the Dollar stores have been navigating for the last several years?",
        'answer': "yes"
    },
    {
        'question': "o Mike, could you elaborate on trends you're seeing from higher income versus middle and lower? I thought that was interesting in your prepared remarks. And maybe just drivers of 3% to 5% comps in the first quarter relative to the 2% comp that you did in the fourth quarter. Have you seen acceleration so far quarter-to-date? And is it traffic or ticket?",
        'answer': "yes"
    },
        {
        'question': "Mike, 2 related things. What are your product priorities, right? When I think about discretionary, particularly around seasonal, right, because you did a lot of new stuff with holiday last year. So priority is there, priorities with regard to multi-price point cooler expansion. And then when you think about comp getting better by maybe 200 basis points, do you think it's equal between discretionary and consumable, each go up by an equal amount? Or is there a difference?",
        'answer': "yes"
    },
            {
        'question': "So just going back to 3.0 format store, just any positive or negative surprises you're seeing with that format? And then as you look forward, what are the bigger opportunities to further optimize the performance?",
        'answer': "yes"
    },
            {
        'question': "On the multi-price, I think you guys said 220 basis points of an uptick. I believe in the second and third quarter, the numbers were higher. Just wondering if you could just speak to that directional change for us. And then on the '25 outlook, on a 3% to 5% comp, you're expecting 50 to 80 basis points of deleverage. Can you just speak to the factors that are going against you? Do you think you get a little bit more leverage on such a great comp?",
        'answer': "yes"
    },
            {
        'question': "Can you remind us how many combo stores you have with Family Dollar and Dollar Tree and how that unwind might look? And just as a follow-up to all the tariff questions, you mentioned that you can mitigate 90%, I think, of the first round of tariffs. Does that mean that the remaining 10% is going to be mitigated with higher prices?",
        'answer': "yes"
    },
            {
        'question': "I think you already started to take some prices up. I'm curious if that was driven by the tariffs, if that was what was driving that decision. And then just going back to Matt Boss' question. What do you assume in that 3% to 5% comp from a traffic versus ticket perspective? I'm not sure if I heard the answer there. And just how much are recent price moves a driver of AUR and ticket? And then just last, when is there a clean break from the TSA? And are there any guarantees of the leases on the Family Dollar stores by Dollar Tree?",
        'answer': "yes"
    },
            {
        'question': "So congratulations on something that has been long awaited by many in the investment community. I have 2 questions. One is, what is the right run rate to think about for Dollar Tree banner on operating margin once we get past transition in 2025? And the second question I had was, anything to call out with respect to a breakup fee in this transaction, if there's anything to point out?",
        'answer': "yes"
    },
            {
        'question': "I wanted to focus on the gross margin. The guidance is for a modest improvement in '25. Now obviously, you said that you mitigated the first 10% tariffs here. I just want to clarify that means that there's no impact on the gross margin. I think that's how the commentary implied, but just wanted to clarify that. And then there should be some tailwinds, so I'm just curious what are some of the other offsets to gross margin this year. If you could give us the puts and takes, that would be helpful.",
        'answer': "yes"
    }
]

pres = """
Good morning, and thank you for joining us today to discuss Dollar Tree's fourth quarter fiscal 2024 results. With me today are Dollar Tree CEO, Mike Creedon; CFO, Jeff Davis; and Chief Transformation Officer, Stewart Glendinning.
Before we begin, I would like to remind everyone that some of the remarks that we will make today about the company's expectations, plans and future prospects are considered forward-looking statements under the safe harbor provision of the Private Securities Litigation Reform Act of 1995. These statements are subject to risks and uncertainties, which could cause actual results to differ materially from those contemplated by our forward-looking statements.
For information on the risks and uncertainties that could affect our actual results, please see the Risk Factors, Business and Management's Discussion and Analysis of Financial Condition and Results of Operations section in our annual report on Form 10-K to be filed on or about March 26, 2025, our most recent press release and Form 8-K and other filings with the SEC.
We caution against reliance on any forward-looking statements made today, and we disclaim any obligation to update any forward-looking statements, except as required by law.
Also during this call, we will discuss certain non-GAAP financial measures. Reconciliations of these non-GAAP items to the most directly comparable GAAP financial measures are provided in today's earnings release available on the IR section of our website. These non-GAAP measures are not intended to be a substitute for GAAP results. Unless otherwise stated, we will refer to our financial results on a GAAP basis.
Additionally, unless otherwise stated, all comparisons discussed today for the fourth quarter of fiscal 2024 are against the same period a year ago. Please note that a supplemental slide deck outlining selected operating metrics is available on the IR section of our website.
Following our prepared remarks, Mike, Jeff and Stewart will take your questions. [Operator Instructions] I'd like to now turn the call over to Mike.
Michael C. CreedonDollar Tree, Inc. · CEO & Director
Thanks, Bob. Good morning, everyone, and thank you for joining our call today. Today is a very exciting day for our company. As I'm sure most of you saw this morning, we announced that Brigade-Macellum will acquire our Family Dollar business for a total price of just over $1 billion.
After a thorough review of our strategic alternatives, the company determined that a sale of Family Dollar is the best way to achieve our value creation goals.
Dollar Tree and Family Dollar are 2 different businesses with limited synergies, and each is at a very different stage of its journey. Separating them will enable each banner to be led and managed by a dedicated team that can focus exclusively on that banner's distinct needs and on realizing each banner's full potential.
Separating will also enable investors to own a business they value more without also having to own a business they value less or that may not fit in their investment profile. It should also make it easier for the market to properly value each business.
Under the terms of the deal, subject to certain closing adjustments, Dollar Tree will receive just over $800 million in cash proceeds. The deal should close in about 90 days, and Family Dollar will remain headquartered here in Chesapeake.
In the fourth quarter, our team was focused on achieving 3 distinct objectives: successfully closing out the year; bringing the strategic review to a favorable conclusion; and setting Dollar Tree on a path to realize its full potential and create long-term value for our associates, customers and shareholders.
With a strong finish to 2024 and the sale of Family Dollar set to close later this year, my leadership team and I will fully dedicate ourselves to Dollar Tree's long-term growth, profitability and returns on capital. Our focus and energy will be devoted to growing sales and profits at this iconic and powerful retail brand.
Dollar Tree offers customers incredible value, convenience and discovery. Our world-class merchants consistently provide our shoppers with an unparalleled and ever-changing assortment of discretionary and consumable products. No other retailer or e-commerce platform can reproduce the immediacy and thrill of that signature Dollar Tree treasure hunt. This is our heritage, and this is our future.
One of our founders, Macon Brock, always spoke of running clean, bright and inviting stores that exceed our customers' expectations. With Dollar Tree as our sole focus, we can remain true to that vision and return to our roots, while still competing and innovating in the marketplace in new and better ways than we could before.
With value, convenience and discovery, Dollar Tree offers just what the customer needs in today's value-seeking environment. By delivering on the fundamentals, we can drive the sales productivity and profitability necessary to create long-term value for our associates, customers and shareholders.
With that, let's now turn to our results. 2024 ended strong as Dollar Tree's multi-price journey continued to build momentum and improvements in store standards and operational efficiency are creating the foundation for sustainable growth and value creation. Fourth quarter results reflect the positive impact of our expanded assortment with our newest multi-price offerings, especially in holiday categories, driving strong year-end sell-through.
In the current economic landscape, we continue to see value-seeking behavior across all customer groups. In recent weeks, many retailers reported that customers, particularly middle-income customers, are shifting towards alternatives that present value. Dollar Tree is also seeing middle-income shoppers who make up about half of our customer base, focusing more on value.
At the same time, we are seeing stronger demand from higher-income customers who increasingly see Dollar Tree as a cost-effective source for an expanding range of products. This trade-in has helped to offset other headwinds.
We believe our ability to continue gaining market share amid such challenging market conditions shows that consumers appreciate the discovery aspect of our unique assortment and our compelling value proposition.
Dollar Tree's Q4 comp was 2%. The quarter got off to a slow start with the late Thanksgiving, but our merchandising teams delivered across the broader holiday season as customers responded positively to our expanded multi-price holiday assortment.
We are particularly gratified that our comp growth was balanced with traffic up 0.7% and ticket up 1.3%. Not only were both positive measures, but ticket actually grew faster than traffic for the first time since Q4 of 2022, during the tail end of the anniversary impact from breaking the dollar.
We are encouraged by the deceleration in consumables mix shift this quarter, which was supported by the strong performance of our expanded holiday assortment. Q4 consumables mix increased 60 basis points to 45.2%, which is an improvement over the average per quarter mix shift of roughly 200 basis points we've seen recently.
Consumables comp was 4.2%, which was on top of a 10.8% comp last year. Discretionary comp was 0.4%, its first positive reading since Q4 of last year.
Multi-price clearly provided a boost to our Q4 performance. And while we are still in the early stages of this journey, I'm pleased with the progress so far and excited about the opportunity still ahead.
With that, let me share some highlights of how our expanded assortment boosted our Q4 results. First, as a reminder, our 3.0 stores are new or converted stores that offer our expanded multi-price assortment throughout the store. Other formats include 2.0, which have a smaller multi-price assortment that is concentrated in a single aisle we call the valley and our 1.0 stores, which are over 95% of the items are still at $1.25.
So in Q4, our in-line 3.0 stores saw a 220 basis point comp lift compared to other formats, including a 40 basis point consumables lift and a 290 basis point discretionary lift. Compared to other formats, 3.0 stores also saw a 20 basis point traffic lift and, more importantly, a 200 basis point ticket lift.
Our merchandising team worked tirelessly to improve and refine execution around our expanded assortment, especially in holiday categories. Across seasonal merchandise broadly, our 3.0 stores saw a 10 percentage point comp lift over other formats, including 30 points in Thanksgiving and 15 points in Christmas.
In everyday categories like textiles, electronics, apparel and toys, we saw comp lifts in the low to mid-teens. Toys, in particular, was a big winner this season. And even in underperforming categories like books, beauty and food, that was more by design as we cut space allocations for these items to make way for more productive categories.
We finished the year with approximately 2,900 3.0 format stores, including roughly 2,600 conversions and 300 new stores. While the number of 3.0 conversions this year fell a bit short of target, we continue to believe it is better to not rush and get them done right with the least amount of disruption for our customers and associates. To that end, we are targeting approximately 5,200 3.0 stores by the end of 2025, including 2,000 new conversions and 300 new stores.
In summary, we're pleased with the first year performance of our expanded assortment in our 3.0 stores. The traffic, ticket and sales lift that we saw is validating the investments we're making in our expanded assortment.
Before I wrap up the Q4 results, I should note that with the decision to sell Family Dollar, from an accounting perspective, our Dollar Tree and Corporate segments are now reported as continuing operations and Family Dollar results are reported as discontinued operations.
Net sales from continuing operations increased 0.7% to $5 billion, reflecting the solid comp performance and strong revenue contribution from noncomp stores, including the former 99 Cents Only portfolio, offset by lapping the 53rd week of last year.
Net sales from discontinued operations decreased 11.2% to $3.3 billion, reflecting Family Dollar's 1.3% comp, the impact of store closings and the lapping of last year's 53rd week. Therefore, on a consolidated basis, net sales were $8.3 billion, which was at the high end of our $8.1 billion to $8.3 billion outlook range.
I'd like to take a few minutes to talk about tariffs and give you a quick supply chain update. As a large retailer and significant importer, we have years of experience dealing with global trade variability. As discussed last quarter, we have multiple contingencies in place to address a variety of tariff scenarios and mitigate the earnings impact of higher tariffs.
These include negotiating supplier cost concessions, changing product specs, dropping noneconomical items, moving country of origin and, lastly, exercising the flexibility multi-price gives us. While we are focused on limiting the financial impact of any new tariffs, we are equally committed to continuously delivering value and market leadership on the items we offer our customers and differentiating ourselves from our competitors across the retail landscape.
Our strategies to diversify country of origin sourcing have been in place for some time now. We intend to remain flexible and nimble, focusing our efforts on sourcing products via channels that deliver the lowest landed cost to us in order to maintain value continuity for our customers.
This includes the optionality to shift sourcing to and from different countries within a relatively short time frame. For example, given our anticipated 2025 imports, the expected net impact of the 10% China tariff that was announced on February 4 prior to any mitigation efforts would have been about $15 million to $20 million per month. Based on our mitigation efforts to date, we have offset more than 90% of this incremental cost, which is reflected in our current 2025 outlook.
With respect to the additional tariffs proposed in March, which included an additional 10% on goods from China and 25% on goods from Canada and Mexico, we believe our potential pre-mitigation exposure is approximately $20 million per month.
As we speak, our merchants are working to mitigate the impact of this latest round of tariffs. On top of that, we are evaluating the potential impact of any additional tariffs that could materialize and impact our sourcing efforts. We have not reflected the impact of this second round of tariffs in our 2025 outlook as the net impact will depend on the eventual policy and the degree, scope and timing of our mitigation efforts.
The imposition of this year's tariffs has introduced uncertainty and volatility. But over the long term, we believe that our mitigation efforts can help us prevent sustained margin erosion.
Finally, concerning our supply chain operations, we will be replacing the DC capacity we lost in Marietta, Oklahoma, and we'll communicate our plans to you once they are finalized. In the interim, we will continue incurring additional stem mile and other related costs until a replacement is up and running.
As an immediate step to help ease some of our current network pressure and support our growing store base, prior to the closing of the sale, we plan to convert the Family Dollar distribution center in Odessa, Texas to a Dollar Tree distribution center.
In sum, we finished 2024 on a high note with strong execution at Dollar Tree. Our results reflected sales momentum powered by growing consumer acceptance of our expanded assortment. With the pending sale of Family Dollar, I am excited at the opportunity to return to Dollar Tree's roots and begin to unlock the full potential of this iconic retail brand.
Before I turn the call over to Jeff to go through the details of our Q4 results, I'd like to welcome Stewart Glendinning to our team. We recently announced that Stewart will take over as CFO at the end of March. Stewart joined the company earlier this year as our Chief Transformation Officer, where he has been heavily immersed in the Family Dollar sale process and charting the future course for Dollar Tree as a standalone organization. After Jeff's Q4 recap, I've asked Stewart to share our 2025 outlook.
Finally, I want to thank Jeff Davis for his partnership these past 2.5 years and for helping to ensure a smooth transition as Stewart assumes his new role. And with that, I'll turn the call over to Jeff.
Jeffrey A. DavisDollar Tree, Inc. · Chief Financial Officer
Thank you, Mike, and good morning. I also want to extend my congratulations to Stewart. Having had the opportunity to work with Stewart over the past few months, I want to echo Mike's comments regarding what has been a smooth transition.
Let me start with an overview of the changes in our financial reporting this quarter. Last June, we initiated a formal review of strategic alternatives for Family Dollar, which ultimately resulted in the transaction we announced today. That process officially ended in the fourth quarter with the company's decision to pursue a sale. And accordingly, Family Dollar was classified as discontinued operations.
As such, our fourth quarter and full year 2024 results are reported on a continuing operations basis, which includes the results of the Dollar Tree segment and Corporate, support and Other. Family Dollar's results are reported as discontinued operations.
In our earnings release this morning, we provided schedules of our 2024 quarterly and annual GAAP and non-GAAP results on a continuing, discontinued and consolidated basis. Unless otherwise stated, my comments today reference our adjusted results from continuing operations.
Also for comparability purposes, please keep in mind that Q4 and FY 2023 included a 53rd week, which positively impacted revenue by approximately $560 million and adjusted EPS by approximately $0.35.
Fourth quarter adjusted EPS was $2.11 from continuing operations and $0.18 from discontinued operations for a total adjusted enterprise EPS of $2.29, which compares to our outlook range of $2.10 to $2.30.
Enterprise results include a $19 million or $0.07 per share benefit from a Mastercard settlement in discontinued ops and a $25 million or $0.09 per share charge for an antidumping duty recorded in continuing operations. Neither of these items were contemplated in our fourth quarter outlook. After netting both items, enterprise adjusted EPS would have been $0.01 above the high end of our outlook range.
Turning to results from continuing operations. Adjusted operating income was $628 million, a 15% decrease from last year. Adjusted operating margin declined 230 basis points as gross margin declined 130 basis points and adjusted SG&A rate increased 100 basis points.
Our adjusted effective tax rate was 24.8% compared to 23.8%, reflecting higher nondeductible expenses for executive compensation and lower work opportunity tax credits in the current year.
Adjusted net income was $455 million compared to $544 million. Adjusted EPS from continuing operations was $2.11, which includes the $0.09 impact from the antidumping duty.
Now let me move to our fourth quarter results for the Dollar Tree segment. Adjusted operating income declined 12.1% to $768 million. Adjusted operating margin declined by approximately 220 basis points, reflecting a 130 basis point decline in gross margin and a 90 basis point increase in adjusted SG&A rate.
Gross margin declined primarily from the loss of sales leverage, lower mark-on and higher shrink, distribution and markdown costs, partially offset by lower freight. Also note that Q4 cost of sales included the $25 million antidumping duty.
Adjusted SG&A rate rose principally from higher depreciation and utilities expense and the loss of sales leverage, which was partially offset by lower general liability claims adjustments.
Moving on to the balance sheet and free cash flow. On a continuing operations basis, total inventory increased $176 million to $2.7 billion on higher mark-on and inventory receipts as we expanded our multi-price assortment. We ended the year with $1.3 billion in cash and cash equivalents.
On the cash flow statement for continuing operations, on a full year basis, we generated $2.2 billion in cash from operating activities, had capital expenditures of $1.3 billion and delivered $893 million of free cash flow.
We ended the year with no borrowings under our revolver, no commercial paper outstanding and bank-defined leverage below 2.5x. Last week, we extended the maturity of our $1.5 billion long-term revolving credit facility to 2030 from 2026. Additionally, we closed on a new $1 billion 364-day revolver ahead of the May maturity of our $1 billion 4% senior notes.
We believe we have ample funding capacity between cash on hand and availability under these credit facilities to meet our near-term debt obligations and provide for the ongoing capital needs of the business.
We did not repurchase any shares in the fourth quarter. For the full year, we repurchased approximately 3.3 million shares of common stock for approximately $404 million, including excise tax. At the end of the year, we had approximately $952 million remaining under our existing share repurchase program.
And now let me turn the call over to Stewart.
Stewart Glendinning
Thank you, Jeff.
Now let me provide our current perspective on fiscal 2025. With the pending sale of Family Dollar, 2025 will be a transitional year for Dollar Tree as a standalone business. We will be working to separate Family Dollar while simultaneously focusing on driving growth and operating improvements in Dollar Tree.
Prior to the closing of the sale, continuing operations, or what we are calling RemainCo, will be burdened with the full cost of corporate shared services. After the sale closes, which we expect will be in June 2025, a transition services agreement, or TSA, will go into effect. This agreement will help offset the shared cost burden until such time as these costs are fully transitioned to the new owners, a process which should unfold throughout 2025 and into 2026.
Looking forward to fiscal year 2025, we expect strong top line growth from the Dollar Tree banner, with sales being positively impacted by multi-price expansion, operating improvements in our stores, new store growth and the continuing ramp-up of our recently opened stores, especially the former 99 Cents Only portfolio.
Taking all this into consideration, we expect fiscal year 2025 sales will be in the range of $18.5 billion to $19.1 billion based on comparable store sales growth of 3% to 5%. We expect a modest improvement in gross margin based on the mitigation actions we've taken to date on implemented tariffs.
That said, the tariff situation remains volatile, and any additional tariffs or unforeseen waterfall effects from the already announced tariffs could affect this assumption. Keep in mind Mike's comments about our ability to mitigate those tariffs over time.
Outside of tariffs, we're forecasting favorability in mark-on, markdown and freight, with a partial offset from higher distribution costs related to incremental D&A from our supply chain investments and additional stem mile and other costs from losing the Marietta DC. Our freight cost outlook is positive across ocean and both inbound and outbound ground.
For SG&A, I'll talk about Dollar Tree segment separately from Corporate, support and Other. Dollar Tree's adjusted SG&A rate in 2024 was 23.8%. In 2025, we expect deleverage of approximately 50 to 80 basis points. This is coming from higher store payroll related to our investments in additional hours and state-mandated minimum wage increases, management incentive compensation, D&A related to our elevated 2024 and 2025 CapEx investments as well as repair and maintenance, as we continue to improve store standards.
Our corporate adjusted SG&A in 2024 was approximately $550 million. We expect this to grow by approximately 20% in 2025. The largest contributor to the year-over-year increase is IT spending, as we move systems off legacy platforms and onto the cloud, followed by payroll from merit increases and incentive comp and D&A.
Under the TSA that I mentioned earlier, we expect to receive approximately $95 million in the last 6 months of 2025 for services provided to Family Dollar over the second half of the year and a similar amount next year. While we will receive TSA income in connection with the cost of supporting Family Dollar for the second half of the year, we will incur these costs over the entire year. This negatively impacts our adjusted EPS by approximately $0.30 to $0.35, given the expected timing of the deal closing.
On a normalized basis, had we received TSA payments for the full year, our net corporate costs at RemainCo would be lower, and our adjusted EPS would be $0.30 to $0.35 higher than the outlook we are providing.
Over the next several years, we expect absolute Corporate SG&A dollars will go down, as overhead costs permanently shift to Family Dollar's new owners. Our Corporate infrastructure adjusts to the level needed to properly support a single banner, stranded costs go away and the TSA runs its course. We are targeting a reduction in our adjusted Corporate SG&A rate of approximately 100 basis points over the medium term.
Finishing out the P&L, we expect net interest and other income of approximately $115 million, an effective tax rate of approximately 25.2% and 216 million shares outstanding, which does not reflect any share repurchases.
Adjusted EPS from continuing operations is expected to be in the range of $5 to $5.50, which compares to last year's $5.10. Capital expenditures are expected to be in the range of $1.2 billion to $1.3 billion, including approximately 400 new Dollar Tree store openings.
With respect to cash, we started the year with $1.3 billion on the balance sheet and expect to receive approximately $800 million of net proceeds from the Family Dollar sale. On top of this, we expect tax benefits from losses on the sale to be approximately $350 million accretive on a cash flow basis.
Our capital allocation priorities remain investing and growing the business, then returning excess cash to shareholders with our preferred vehicle to date having been share repurchase. It is reasonable to assume that we will be back in the market repurchasing shares this year.
Our balance sheet is strong. For this reason, we expect to come to market with a new debt offering following our May debt maturity and the closing of the Family Dollar sale.
In the near term, we expect first quarter net sales to be in the range of $4.5 billion to $4.6 billion based on comparable net sales growth in the 3% to 5% range and adjusted diluted earnings per share in the range of $1.10 to $1.25.
In summary, 2025 is going to be a transitional year. With the sale of Family Dollar, we can fully focus on unlocking value at Dollar Tree. We had a solid 2024, and we feel good about many parts of our business heading into 2025.
We're optimistic about the top line, and we're addressing cost pressures on several fronts, with tariffs being at the top of that list. We feel great about our cash position and our ability to generate meaningful levels of cash going forward.
With a solid balance sheet and prudent CapEx commitments, we should have the ability to return a substantial amount of capital to shareholders this year and into the future.
With that, I'll turn the call back over to Mike.
Michael C. CreedonDollar Tree, Inc. · CEO & Director
Thanks, Stewart. This is a pivotal time at Dollar Tree. We are shifting our long-term operational focus away from an integrated 2-banner model. With the sale of Family Dollar, our leadership team can focus all our energy and resources on growing Dollar Tree.
I strongly believe selling Family Dollar and returning to our roots with an expanded assortment at Dollar Tree has created material value. Dollar Tree remains one of the best growth stories in retail, and the separation of the 2 businesses will allow us to move forward with a single-minded focus of driving growth and profitability.
As Stewart indicated, 2025 is going to be a transition year as we pivot to operating Dollar Tree as a standalone entity. After 2025, on a go-forward basis, we will drive top line results by growing comp and opening new stores. We expect increased sales productivity will allow us to expand gross margin, begin leveraging SG&A and grow EPS. Most importantly, by returning a meaningful level of cash to our shareholders, we can leverage that EPS growth even more.
I'd like to close with a shout-out to the entirety of the Dollar Tree and Family Dollar teams. They have worked masterfully and tirelessly on behalf of our shareholders and our customers. I could not be more honored to count myself among them.
And with that, we're ready to take your questions.
"""
